In [287]:
# this code is inspired by this kernel: https://www.kaggle.com/tapioca/lightgbm-score-booster
import numpy as np
import pandas as pd
import time
import lightgbm as lgb
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [292]:
# load merged data (by assetCodes) from local
merged_data = pd.read_csv("./merged_data_assetCodes.csv") # success rows: 1147200
# delete the first 10 trading days that returnsPrev10 are all NaN
# To Fang: besides the first 10 trading days, can you clean other NaN cells in merged_data?
# converting 'time' column into datetime format?
merged_data = merged_data.loc[merged_data['time'] >= '2007-02-15']
merged_data

,time,assetCode,assetName,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,universe,companyCount,relevance,sentimentNegative,sentimentNeutral,sentimentPositive,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
14290,2007-02-15,A.N,Agilent Technologies Inc,4095135.0,33.83,32.990,0.027019,0.004262,0.026171,-0.001572,0.050947,0.025490,0.039098,0.007461,-0.029993,1.0,1.000000,1.000000,0.183994,0.564769,0.251237,2.153846,2.153846,2.153846,2.153846,2.153846,8.153846,8.153846,10.153846,18.153846,19.153846
14291,2007-02-15,AAI.N,AirTran Holdings Inc,1378650.0,11.67,11.650,-0.002564,0.003445,-0.003326,-0.001498,0.049460,0.051444,0.006601,-0.010884,-0.013111,0.0,1.800000,0.906066,0.129506,0.597227,0.273268,0.800000,0.800000,0.800000,0.800000,0.800000,5.400000,6.100000,8.000000,8.400000,8.400000
14292,2007-02-15,AAP.N,Advance Auto Parts Inc,3884400.0,38.75,37.000,0.046731,-0.015957,0.046114,-0.019388,0.033058,-0.026059,0.033965,-0.026448,-0.028244,1.0,1.111111,1.000000,0.173526,0.570563,0.255912,3.222222,3.222222,3.222222,3.222222,3.277778,11.111111,12.000000,13.055556,13.111111,14.111111
14293,2007-02-15,AAPL.O,Apple Inc,12997017.0,85.21,85.310,-0.001055,0.008035,-0.002337,0.000738,0.005546,-0.010669,-0.020233,-0.044809,-0.014505,1.0,1.000000,1.000000,0.027258,0.353779,0.618963,1.000000,1.000000,1.000000,1.000000,1.000000,5.666667,6.333333,33.666667,68.666667,72.166667
14294,2007-02-15,ABB.N,ABB Ltd,10168100.0,18.48,18.245,-0.033978,-0.022240,-0.035524,-0.026040,0.025527,0.013048,0.009882,-0.010927,0.017172,1.0,1.538462,0.919368,0.091920,0.332099,0.575981,0.923077,0.923077,0.923077,0.923077,1.000000,7.692308,7.692308,11.692308,12.692308,13.692308
14295,2007-02-15,ABC.N,AmerisourceBergen Corp,1683900.0,55.32,54.180,0.024444,-0.004044,0.023468,-0.007728,0.056330,0.033969,0.066111,0.048016,0.022841,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14296,2007-02-15,ABD.N,ACCO Brands Corp,570000.0,22.86,22.920,-0.001747,-0.009935,-0.002499,-0.012493,-0.032586,-0.050145,-0.037874,-0.060766,-0.046800,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14297,2007-02-15,ABG.N,Asbury Automotive Group Inc,449000.0,25.84,25.750,0.013333,0.024264,0.012451,0.017546,0.051689,0.052739,NaN,NaN,0.046899,0.0,1.000000,1.000000,0.106947,0.618299,0.274755,3.666667,3.666667,3.666667,3.666667,3.666667,12.666667,12.666667,12.666667,12.666667,12.666667
14298,2007-02-15,ABM.N,ABM Industries Inc,183900.0,26.86,26.620,0.002987,-0.007827,0.002991,-0.007520,0.025582,0.027799,0.014998,0.008337,0.042174,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14299,2007-02-15,ABT.N,Abbott Laboratories,4148300.0,52.80,52.580,0.001898,-0.002277,0.001556,-0.000848,-0.001324,0.001524,0.012771,0.018683,-0.012228,1.0,4.000000,0.192579,0.308462,0.353499,0.338039,0.250000,0.250000,0.250000,0.250000,0.250000,2.500000,4.000000,7.500000,8.500000,9.000000


In [301]:
# feature engineering
merged_data['daytrend'] = merged_data['close'] / merged_data['open']
merged_data['average'] = (merged_data['close'] + merged_data['open']) / 2
merged_data['moving_average_10_days'] = merged_data.groupby('assetCode')['close'].transform(lambda x: x.rolling(window=10).mean())

# merged_data['pricevolume'] = merged_data['volume'] * merged_data['close']

In [302]:
# extract successfully merged rows
merged_success = merged_data[~np.isnan(merged_data['sentimentPositive'])].reset_index(drop=True)

In [252]:
# generate a random subsample from merged_success ...
#len_sub = 10000
#print('generate a random subsample of length ' + str(len_sub) + ' from merged_success of length ' + str(len(merged_success)) + ' ...')
#random_rows = np.random.choice(len(merged_success), len_sub).tolist()
#random_rows.sort()
#mdf = merged_success.loc[random_rows].reset_index(drop=True)

In [304]:
# ... Or, use the entire data 'merged_success'/'merged_data'
mdf = merged_success
# mdf = merged_data

In [305]:
#########################
print('building training set...')
print('-------------------------')
targetcols = ['target']
mdf[targetcols[0]] = (mdf['returnsOpenNextMktres10'] > 0).astype(int) # we be classifying binary variable
traincols = [col for col in mdf.columns if col not in ['time', 'assetCode', 'assetName', 'universe', 'returnsOpenNextMktres10'] + targetcols]

dates = mdf['time'].unique()
print('The data covers ' + str(len(dates)) + ' unique dates.')
print('We assign data of the first 60% dates as the training set,...')
print('the next 20% dates as the validation set,...')
print('and the last 20% dates as the test set.')
train = range(len(dates))[:int(0.60*len(dates))]
val = range(len(dates))[int(0.60*len(dates)):int(0.80*len(dates))]
test = range(len(dates))[int(0.80*len(dates)):]

building training set...
-------------------------
The data covers 2488 unique dates.
We assign data of the first 60% dates as the training set,...
the next 20% dates as the validation set,...
and the last 20% dates as the test set.


In [306]:
# train data
# Shall we clean NaN in the pre_processing steps?
Xt = mdf[traincols].fillna(0).loc[mdf['time'].isin(dates[train])].values
Yt = mdf[targetcols].fillna(0).loc[mdf['time'].isin(dates[train])].values

# validation data
Xv = mdf[traincols].fillna(0).loc[mdf['time'].isin(dates[val])].values
Yv = mdf[targetcols].fillna(0).loc[mdf['time'].isin(dates[val])].values

print(Xt.shape, Yt.shape, Xv.shape, Yv.shape)

(640622, 29) (640622, 1) (251806, 29) (251806, 1)


In [230]:
# daily score
def daily_score(predtemplatedf): 
    """
    type predtemplatedf: DataFrame (shape: number of assetCode's in one day * 4)
    rtype: float
    """
    return (predtemplatedf['universe'] * predtemplatedf['returnsOpenNextMktres10'] * predtemplatedf['confidenceValue']).sum()

In [231]:
# final submission score
def final_score(score):
    """
    type score: DataFrame (shape: number of days in the test set * 2)
    rtype: float
    """
    return np.mean(score.daily_score) / np.std(score.daily_score)

In [285]:
# for prediction array at each date, rescale it to mean return of zero and std of 0.125
def post_scaling(pred_Y):
    """
    type pred_Y: array
    rtype: array
    """
    mean, std = np.mean(pred_Y), np.std(pred_Y)
    pred_Y = (pred_Y - mean) / std / 8
    return np.clip(pred_Y, -1, 1)

In [307]:
#######################################
##
## LightGBM: training
##
#######################################
print('training LightGBM...')
print('--------------------')
# we use the same parameters as the benchmark kernel
params = {'objective': 'binary',
          'metric': 'binary_logloss',
          'num_leaves': 60,
          'max_depth': -1,
          'learning_rate': 0.01,
          'bagging_fraction': 0.9, # subsample
          'feature_fraction': 0.9, # colsample_bytree
          'bagging_freq': 5,       # subsample_freq
          'bagging_seed': 2018,
          'verbosity': -1}

lgtrain, lgval = lgb.Dataset(Xt, Yt[:,0]), lgb.Dataset(Xv, Yv[:,0])

train_start = time.time()
lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=200)
print('--------------------')
print('training is done!')
print("training LightGBM took %.0f seconds." % (time.time() - train_start))

training LightGBM...
--------------------
Training until validation scores don't improve for 100 rounds.
[200]	training's binary_logloss: 0.685143	valid_1's binary_logloss: 0.686702
[400]	training's binary_logloss: 0.682685	valid_1's binary_logloss: 0.686235
[600]	training's binary_logloss: 0.680819	valid_1's binary_logloss: 0.68624
Early stopping, best iteration is:
[550]	training's binary_logloss: 0.681255	valid_1's binary_logloss: 0.686221
--------------------
training LightGBM took 77 seconds.


In [308]:
#######################################
##
## LightGBM: prediction
##
#######################################
print('generating predictions...')
print('-------------------------')
mdftest = merged_data.loc[merged_data['time'].isin(dates[test])].reset_index(drop=True) # prediction based on merged_data
print(str(len(mdftest)) + ' samples to be predicted...')
score = pd.DataFrame({'time': dates[test], 'daily_score': np.zeros(len(test))})

pred_start = time.time()
for test_day in dates[test]:
    mdftestdaily = mdftest[mdftest['time'] == test_day]
    test_X = mdftestdaily[traincols].fillna(0).values
    pred_Y = lgbmodel.predict(test_X, num_iteration=lgbmodel.best_iteration) * 2 - 1 # transform [0,1] to [-1,1]
    predtemplatedf = mdftestdaily[['assetCode', 'universe', 'returnsOpenNextMktres10']]
    predtemplatedf = predtemplatedf.assign(confidenceValue = post_scaling(pred_Y))
    score.loc[score['time'] == test_day, 'daily_score'] = daily_score(predtemplatedf)
print('prediction is done!')
print('prediction took %.0f seconds.' % (time.time() - pred_start))
print('-------------------------')
submission_score = final_score(score)
print('our submission score is: ', submission_score)

generating predictions...
-------------------------
882206 samples to be predicted...
prediction is done!
prediction took 82 seconds.
our submission score is:  0.5106780727637015


In [283]:
## record of results:
# train merged_success, test merged_success: 0.50
# train merged_data, test merged_success: 0.4964
## Remark: it seems that training entire merged_data makes no improvement compared to training merged_success which saves time
## But, we should test all merged_data not only merged_success!
# train merged_data, test merged_data: 0.4827
# train merged_success, test merged_data: 0.4844 (I'll use this as benchmark to save time for future test)
## feature engineering ...
# adding bartrend: 0.4857
# adding [bartrend, average, pricevolume]: 0.4790
## using new merged_data_assetCodes (115w rows successfully merged compared to 111w of previous version)
# merged_data_assetCodes: 0.4888
## adding post_scaling: 0.5133 # why rescaling improves prediction performance?


In [ ]:
## to-do:
# what're the most important features?
# does including news improve prediction performance?
# can we use two seperate models to predict those assetCode's with and without news data? (only about 1/4 have news)
# classifying more than two categories?
# adding lag features to account for the time series dependence?
# should we rescale the X before training?
# pre-process NaN
# how to train lgbm?
# excluding outliers of 0.5 < daytrend < 1.5